Quantitative Finance Project
By: Varun Gopal, Tyler Dixon, and Abhinav Kakumanu

Objective of this project is to highlight the methods learned in class and apply them with market data.

Our project revolves around examining economic data and building a predictive model that can help guide returns.

In [1]:
# Importing basic files

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime

import pandas_datareader as pdr
import pandas_datareader.famafrench
from functools import reduce

In [2]:
# Get all the datasets 
pandas_datareader.famafrench.get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
ff = pdr.get_data_famafrench('10_Industry_Portfolios', 1926)

In [4]:
type(ff)

dict

In [5]:
ff.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 'DESCR'])

In [6]:
ff.keys()
ten_ind = ff[1]

In [7]:
industry_name = [i for i in ff[0].columns]
industry_name

['NoDur',
 'Durbl',
 'Manuf',
 'Enrgy',
 'HiTec',
 'Telcm',
 'Shops',
 'Hlth ',
 'Utils',
 'Other']

In [8]:
ten_ind = ten_ind.resample('Q').apply(lambda x: (1+ (x/100)).product()-1)

In [9]:
ten_ind

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926Q3,0.062499,0.008370,0.050122,-0.014611,0.057796,0.026294,-0.000549,0.095794,0.048697,0.039668
1926Q4,0.036895,-0.054746,-0.002352,0.094341,-0.030808,0.021030,0.007802,0.077902,0.043239,-0.003773
1927Q1,-0.014018,0.012062,0.058804,-0.018129,0.070875,0.085652,0.029499,0.085339,0.067053,0.130579
1927Q2,0.060628,-0.011549,0.040557,0.029293,0.068964,-0.004983,0.034686,0.149034,0.074695,0.084061
1927Q3,0.108930,0.072260,0.101493,0.041747,0.102346,0.112249,0.131985,0.152781,0.210272,0.068413
...,...,...,...,...,...,...,...,...,...,...
2022Q1,-0.081467,-0.138146,-0.034966,0.482865,-0.126784,-0.016195,-0.093717,-0.166785,0.067288,-0.039082
2022Q2,-0.141088,-0.180388,-0.178501,-0.094793,-0.248041,-0.183191,-0.184645,-0.242632,-0.040033,-0.163498
2022Q3,-0.129309,-0.068174,-0.054447,0.037160,-0.040105,-0.142568,-0.041964,-0.013281,-0.039541,-0.035206


In [10]:
type(ten_ind.index)

pandas.core.indexes.period.PeriodIndex

We have gotten all the data from 1926 for different industries. Here are the industries:
1. Consumer Nondurables: Food, Tobacco, Textiles, Apparel, Leather, Toys
2. Consumer Durables -- Cars, TVs, Furniture, Household Appliances
3. Manufacturing -- Machinery, Trucks, Planes, Chemicals, Off Furn, Paper, Com Printing
4. Oil, Gas, and Coal Extraction and Products
5. Business Equipment -- Computers, Software, and Electronic Equipment
6. Telephone and Television Transmission
7. Wholesale, Retail, and Some Services (Laundries, Repair Shops)
8. Healthcare, Medical Equipment, and Drugs
9. Utilities
10. Other -- Mines, Constr, BldMt, Trans, Hotels, Bus Serv, Entertainment, Finance

The next step will be to gather important economic data from the Federal Reserve and map it alongside the data. 
First question is what do we want to map?

We will do our best to get variables that will not impact one specific industry. However, there may be data that skew towards a specific industry. We will highlight the bias. 

According to https://groww.in/blog/macroeconomic-factors-that-influence-us-stock-markets
1. Gross Domestic Product (GDP)
2. Inflation
3. Unemployment Rate (Payrolls)
4. Retail Sales
5. Industrial Output 

We will also get market data on interest rates, corporate profits, and corporate debt.


Get the data from Federal Reserve Economic Data (FRED)

Another Hypothesis is that markets will react to any economic indicators quickly. Since indicators such as GDP are lagging indicators, we will need market representations (such as yield spreads) that can help understand how stocks react to the propsect of economic events.

First, let us use the common models and then we can use some of the proxies later on.

In [11]:
# RGDP is real GDP... since we will examine inflation later
rgdp = pdr.get_data_fred('GDPC1', 1950)
rgdp = rgdp.pct_change().dropna()
rgdp.index = rgdp.index.to_period('Q')

In [12]:
#CPI to capture inflation
cpi = pdr.get_data_fred('CPIAUCSL',1947)
cpi = cpi.pct_change().dropna()
cpi = cpi.resample('Q').apply(lambda x: (1+ x).product()-1)
cpi.index = cpi.index.to_period('Q')

In [13]:
cpi

,CPIAUCSL
DATE,
1947Q1,0.024209
1947Q2,0.003636
1947Q3,0.034420
1947Q4,0.024956
1948Q1,0.003845
...,...
2022Q1,0.023444
2022Q2,0.025241
2022Q3,0.006145


In [14]:
# Corporate Profits
corp_profit = pdr.get_data_fred('CP', 1947)
corp_profit = corp_profit.pct_change().dropna()
corp_profit.index = corp_profit.index.to_period('Q')


In [15]:
corp_profit

,CP
DATE,
1947Q2,-0.053801
1947Q3,-0.010775
1947Q4,0.091762
1948Q1,0.057369
1948Q2,0.050634
...,...
2021Q4,-0.023466
2022Q1,0.026317
2022Q2,0.074003


In [16]:
# Unemployment Rate
unemp = pdr.get_data_fred('UNRATE', 1948)
unemp = unemp.resample('Q').mean()
unemp.index = unemp.index.to_period('Q')

In [17]:
unemp

,UNRATE
DATE,
1948Q1,3.733333
1948Q2,3.666667
1948Q3,3.766667
1948Q4,3.833333
1949Q1,4.666667
...,...
2022Q1,3.800000
2022Q2,3.600000
2022Q3,3.566667


In [18]:
# Interest Rates (10 yr treasury rates)
irate = pdr.get_data_fred('FEDFUNDS', 1953)
irate = irate.pct_change().dropna()
irate = irate.resample('Q').apply(lambda x: (1+x).product()-1)
irate.index = irate.index.to_period('Q')

In [19]:
irate

,FEDFUNDS
DATE,
1954Q3,0.337500
1954Q4,0.196262
1955Q1,0.054688
1955Q2,0.214815
1955Q3,0.329268
...,...
2022Q1,1.500000
2022Q2,5.050000
2022Q3,1.115702


In [20]:
type(irate)

pandas.core.frame.DataFrame

We need to put the data together into a table. Additionally, we will need to group by quarters.

In [21]:
dfs = [ten_ind, rgdp, cpi, corp_profit, unemp, irate]

In [38]:
merged_df = ten_ind
for i in range(len(dfs)-1):
    if dfs[i].index[0] > dfs[i+1].index[0]:
        merged_df = merged_df.merge(dfs[i+1], how = 'left' , left_index= True, right_index = True)
    else:
        merged_df = merged_df.merge(dfs[i+1], how = 'right' , left_index= True, right_index = True)
# Since there was not as much data for Q1 2023, we excluded that from the dataset
merged_df = merged_df[:-1]

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,GDPC1,CPIAUCSL,CP,UNRATE,FEDFUNDS
DATE,,,,,,,,,,,,,,,
1954Q3,0.095295,0.095394,0.113667,0.105200,0.106222,0.148205,0.137887,0.109301,0.063919,0.123850,0.011293,-0.004826,0.056993,5.966667,0.337500
1954Q4,0.131423,0.146668,0.213528,0.180764,0.209471,0.263143,0.114878,0.213420,0.052111,0.259460,0.019567,-0.001492,0.071474,5.333333,0.196262
1955Q1,0.039101,0.075081,0.067013,0.043564,0.088119,0.088999,0.022833,0.061326,0.046011,0.073101,0.028562,0.000747,0.120243,4.733333,0.054688
1955Q2,0.036104,0.065560,0.077438,0.083466,0.069817,0.044356,0.081077,0.067599,0.043439,0.076952,0.016272,-0.002986,0.016828,4.400000,0.214815
1955Q3,-0.002395,0.047865,0.018076,-0.005216,-0.058165,-0.024455,0.022043,-0.003914,0.011364,-0.021357,0.013505,0.005241,0.026803,4.100000,0.329268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021Q4,0.019324,-0.039307,0.040732,-0.039813,-0.032828,-0.117818,0.023229,-0.179456,0.103418,0.021902,0.016957,0.025030,-0.023466,4.200000,0.000000
2022Q1,-0.081467,-0.138146,-0.034966,0.482865,-0.126784,-0.016195,-0.093717,-0.166785,0.067288,-0.039082,-0.004103,0.023444,0.026317,3.800000,1.500000
2022Q2,-0.141088,-0.180388,-0.178501,-0.094793,-0.248041,-0.183191,-0.184645,-0.242632,-0.040033,-0.163498,-0.001446,0.025241,0.074003,3.600000,5.050000


Begin running the regressions to predict the sensitivity of markets to other events.

In [23]:
# import sklearn
from sklearn.linear_model import LinearRegression

In [27]:
# Create a separate list with all the independent variables
ind_var = []
for col in merged_df.columns[10:]:
    ind_var += [col]


['GDPC1', 'CPIAUCSL', 'CP', 'UNRATE', 'FEDFUNDS']

In [40]:
X = merged_df[ind_var]


,GDPC1,CPIAUCSL,CP,UNRATE,FEDFUNDS
DATE,,,,,
1954Q3,0.011293,-0.004826,0.056993,5.966667,0.337500
1954Q4,0.019567,-0.001492,0.071474,5.333333,0.196262
1955Q1,0.028562,0.000747,0.120243,4.733333,0.054688
1955Q2,0.016272,-0.002986,0.016828,4.400000,0.214815
1955Q3,0.013505,0.005241,0.026803,4.100000,0.329268
...,...,...,...,...,...
2021Q4,0.016957,0.025030,-0.023466,4.200000,0.000000
2022Q1,-0.004103,0.023444,0.026317,3.800000,1.500000
2022Q2,-0.001446,0.025241,0.074003,3.600000,5.050000


We will now run the regression. We will use a for loop and try to store the values into a dictionary so we can then extract those values easily.

In [64]:
coef_dicts = {}
model = LinearRegression()

# for i in industry_name:
#     print(merged_df[i])

for i in industry_name:
    model.fit(X,merged_df[i])
    coef_dicts[i] = model.coef_
        

In [65]:
coef_dicts

{'NoDur': array([ 0.38217155, -0.69386653,  0.11953326,  0.01848942, -0.02761817]),
 'Durbl': array([-0.05252822, -1.67319151,  0.30037546,  0.02505912, -0.03220324]),
 'Manuf': array([ 0.49399641, -0.32602931,  0.17697471,  0.01687868, -0.02774935]),
 'Enrgy': array([-1.29274219,  3.69713087,  0.37657799,  0.00750023, -0.01359115]),
 'HiTec': array([ 0.18464173, -1.23291101,  0.26735113,  0.01702601, -0.04944891]),
 'Telcm': array([ 0.24615491, -0.95571421,  0.12667126,  0.02046081, -0.03537414]),
 'Shops': array([-0.10426935, -1.05355986,  0.24483821,  0.02060039, -0.03618558]),
 'Hlth ': array([-1.04980364, -1.15532357,  0.14233581,  0.0158724 , -0.04717332]),
 'Utils': array([ 0.24395242, -0.07748587, -0.1614371 ,  0.00936603, -0.00480924]),
 'Other': array([ 0.77934541, -1.01116652,  0.07167402,  0.0148557 , -0.02346003])}